In [1]:
# Importando bibliotecas
import requests
import json
import pandas as pd
import time
import os
from datetime import datetime
import psycopg2

from dotenv import load_dotenv

load_dotenv()

TINY_TOKEN = os.getenv("TINY_TOKEN")
HOST = os.getenv("HOST")
POSTGRES_DB = os.getenv("POSTGRES_DB")
POSTGRES_USER = os.getenv("POSTGRES_USER")
POSTGRES_PASSWORD = os.getenv("POSTGRES_PASSWORD")

# Informações de conexão com o banco de dados PostgreSQL
db_config = {
    "host": 'localhost',
    "database": POSTGRES_DB,
    "user": POSTGRES_USER,
    "password": POSTGRES_PASSWORD,
}


pd.set_option('display.max_columns', None)

# Registra o tempo antes da execução
start_prog = time.time()

# Configurações da API Tiny

token = TINY_TOKEN
formato = 'JSON'

In [2]:
# Carregando IDs de produtos

conn = psycopg2.connect(**db_config) # Conecta ao banco de dados usando db_config

query = "select tiny_id from tiny_products;"

# tiny_id_df = pd.read_sql(query, db_config)
df_tiny_id_original = pd.read_sql_query(query, conn)

conn.close()

df_tiny_id_original.shape

C:\Users\couti\AppData\Local\Temp\ipykernel_684\1680147997.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_tiny_id_original = pd.read_sql_query(query, conn)


(482, 1)

In [3]:
# Salve o DataFrame original
df_tiny_id = df_tiny_id_original.copy()

# Agora, df conterá apenas a coluna especificada
df_tiny_id.shape

(482, 1)

In [4]:
# Remova as linhas com valores NaN - Linha 378
df_tiny_id_no_nan = df_tiny_id.dropna(subset=['tiny_id'])

df_tiny_id_no_nan.shape

(482, 1)

In [5]:
# Verifica se há duplicatas
duplicatas = df_tiny_id_no_nan.duplicated()
numero_de_duplicatas = duplicatas.sum()

print(f'Número de duplicatas: {numero_de_duplicatas}')

# # Encontre as duplicatas no DataFrame
# duplicatas = df_tiny_id_no_nan[df_tiny_id_no_nan.duplicated(keep=False)]

# # Exiba as próprias duplicatas
# duplicatas

Número de duplicatas: 0


In [6]:
# Remove as duplicatas e atualiza o DataFrame
df_tiny_id_no_dup = df_tiny_id_no_nan.drop_duplicates()

df_tiny_id_no_dup.shape

(482, 1)

In [7]:
# Requisitando produtos à API Tiny

start_time = time.time() # Registra o tempo antes da execução

url = 'https://api.tiny.com.br/api2/produto.obter.estoque.php'

# df_tiny_id_no_dup = df_tiny_id_no_dup.head(3)
#df_tiny_id = pd.read_csv('tiny_id_no_dup.csv')
df_tiny_id = df_tiny_id_no_dup

responses = []  # Lista para armazenar os resultados
num = 0

def enviarREST(url, data, optional_headers=None):
    headers = optional_headers if optional_headers is not None else {}

    response = requests.post(url, data=data, headers=headers)

    if response.status_code != 200:
        raise Exception(f"Problema com {url}, Status Code: {response.status_code}")

    return response.text


for id in df_tiny_id['tiny_id']:
    print(f'Buscando dados de {id}')
    num +=1
    print(f'Loop: {num}')
    data = {'token': token, 'id': id, 'formato': formato}
    response = enviarREST(url, data)
    responses.append(response)

    # Verifica se é múltiplo de 50 para aguardar 1min a cada 50 requisições
    if num % 50 == 0:
        print("Esperando 1 minuto...")
        time.sleep(60)  # Pausa por 1 minuto

# Registra o tempo depois da execução
end_time = time.time()

# Calcula o tempo decorrido
elapsed_time = end_time - start_time

print(f"\n\nTempo :decorrido {elapsed_time / 60} minutos")
print(f"Total de resultados encontrados: {len(responses)}")

Buscando dados de 509517168
Loop: 1
Buscando dados de 598796832
Loop: 2
Buscando dados de 565287467
Loop: 3
Buscando dados de 565409748
Loop: 4
Buscando dados de 566317152
Loop: 5
Buscando dados de 730206769
Loop: 6
Buscando dados de 565409173
Loop: 7
Buscando dados de 730206884
Loop: 8
Buscando dados de 566289523
Loop: 9
Buscando dados de 566315216
Loop: 10
Buscando dados de 566279845
Loop: 11
Buscando dados de 599250891
Loop: 12
Buscando dados de 506792227
Loop: 13
Buscando dados de 565663877
Loop: 14
Buscando dados de 565665071
Loop: 15
Buscando dados de 596827212
Loop: 16
Buscando dados de 509411237
Loop: 17
Buscando dados de 561593498
Loop: 18
Buscando dados de 509359755
Loop: 19
Buscando dados de 506825032
Loop: 20
Buscando dados de 506333260
Loop: 21
Buscando dados de 753620478
Loop: 22
Buscando dados de 753620554
Loop: 23
Buscando dados de 558025105
Loop: 24
Buscando dados de 558020144
Loop: 25
Buscando dados de 565286240
Loop: 26
Buscando dados de 506979980
Loop: 27
Buscando d

In [20]:
responses

['{"retorno":{"status_processamento":"3","status":"OK","produto":{"id":"509517168","nome":"Cabo P10 X P10 Santo Angelo Ninja 10ft - 3.05m","codigo":"7899028808537","unidade":"Un","saldo":53,"saldoReservado":1,"depositos":[{"deposito":{"nome":"Cristiano","desconsiderar":"N","saldo":0,"empresa":"mcenter168"}},{"deposito":{"nome":"M A Bueno","desconsiderar":"N","saldo":0,"empresa":"mcenter168"}},{"deposito":{"nome":"M A Lang","desconsiderar":"N","saldo":0,"empresa":"mcenter168"}},{"deposito":{"nome":"Mundo das Teclas","desconsiderar":"N","saldo":-100,"empresa":"mcenter168"}},{"deposito":{"nome":"Musical Filial","desconsiderar":"N","saldo":98,"empresa":"mcenter168"}},{"deposito":{"nome":"Musical Matriz","desconsiderar":"N","saldo":55,"empresa":"mcenter168"}},{"deposito":{"nome":"Talyta","desconsiderar":"N","saldo":0,"empresa":"mcenter168"}}]}}}',
 '{"retorno":{"status_processamento":"3","status":"OK","produto":{"id":"598796832","nome":"Cabo P10 X P10L Santo Angelo Ninja L 10ft - 3.05m","co

In [8]:
# Gravando respostas em txt
datetime_now = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

file_path = f"../Data/Output/tiny_estoque_json_{datetime_now}.txt"

with open(file_path, 'w') as file:
    for line in responses:
        file.write(line + '\n')  # Adicione uma quebra de linha

print(f'Os dados foram gravados no arquivo: {file_path}')

Os dados foram gravados no arquivo: ../Data/Output/tiny_estoque_json_2023-11-01_15-44-09.txt


In [9]:
# Verificandos e há erros de processamento
text_search1 = '"status_processamento":"1"'
text_search2 = '"status_processamento":"2"'

lines_search1 = []
lines_search2 = []

with open(file_path, 'r') as file:
    for n_line, line in enumerate(file, start=1):
        if text_search1 in line:
            lines_search1.append(n_line)
        if text_search2 in line:
            lines_search2.append(n_line)

if lines_search1 and lines_search2:
    print(f'"status_processamento":"1" foi encontrado nas linhas: {lines_search1}')
    print(f'"status_processamento":"2" foi encontrado nas linhas: {lines_search2}')
elif lines_search1:
    print(f'"status_processamento":"1" foi encontrado nas linhas: {lines_search1}')
    print('"status_processamento":"2" não foi encontrado em nenhuma linha.')
elif lines_search2:
    print(f'"status_processamento":"2" foi encontrado nas linhas: {lines_search2}')
    print('"status_processamento":"1" não foi encontrado em nenhuma linha.')
else:
    print('Não foi verificado erro de processamento"')


Não foi verificado erro de processamento"


# Df Base

In [14]:
# lista de JSONs
lista_json = responses

# Inicialize o DataFrame vazio
tiny_estoque_df = pd.DataFrame()

# Processar cada JSON na lista
for json_str in lista_json:
    # Transforme a string JSON em um objeto Python
    json_data = json.loads(json_str)

    # Extrair a parte "produto" do JSON
    produto = json_data["retorno"]["produto"]
    depositos = produto['depositos']

    # Verifique se a lista de depósitos está presente
    if depositos:
        # Crie um DataFrame temporário a partir da lista de depósitos
        temp_df = pd.json_normalize(depositos)

        # Adicione as colunas do nível "produto" ao DataFrame temporário
        for key, value in produto.items():
            temp_df[key] = value

        # Concatene o DataFrame temporário ao DataFrame principal
        # produto_df = pd.concat([produto_df, temp_df], ignore_index=True)
        tiny_estoque_df = pd.concat([tiny_estoque_df, temp_df], ignore_index=True)

# Reordene as colunas, se necessário
col_order = [
    'id', 'nome', 'codigo', 'unidade', 'saldo', 'saldoReservado',
    'deposito.nome', 'deposito.desconsiderar', 'deposito.saldo', 'deposito.empresa'
]
tiny_estoque_df = tiny_estoque_df[col_order]

# O DataFrame 
tiny_estoque_df.head(1)
tiny_estoque_df.shape


(3452, 10)

In [13]:
# Verifica se há duplicatas
duplicatas = tiny_estoque_df.duplicated()
total_duplicatas = duplicatas.sum() 

print(f'Número de duplicatas: {total_duplicatas}')

# Encontra duplicatas no DataFrame
duplicatas = tiny_estoque_df[tiny_estoque_df.duplicated(keep=False)]

# Exiba duplicatas
duplicatas['id']

l_duplicatas = duplicatas[duplicatas['id'] == '565663877']
l_duplicatas.shape

Número de duplicatas: 0


(0, 10)

In [17]:
tiny_estoque_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3452 entries, 0 to 3451
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   id                      3452 non-null   object
 1   nome                    3452 non-null   object
 2   codigo                  3452 non-null   object
 3   unidade                 3452 non-null   object
 4   saldo                   3452 non-null   int64 
 5   saldoReservado          3452 non-null   int64 
 6   deposito.nome           3452 non-null   object
 7   deposito.desconsiderar  3452 non-null   object
 8   deposito.saldo          3452 non-null   int64 
 9   deposito.empresa        3452 non-null   object
dtypes: int64(3), object(7)
memory usage: 269.8+ KB


In [20]:
unique_values = tiny_estoque_df['deposito.empresa'].unique()
print(unique_values)


['mcenter168' 'cpmusical']


# Salvando na base de dados

In [18]:
tiny_estoque_df.columns

Index(['id', 'nome', 'codigo', 'unidade', 'saldo', 'saldoReservado',
       'deposito.nome', 'deposito.desconsiderar', 'deposito.saldo',
       'deposito.empresa'],
      dtype='object')

In [23]:
conn = psycopg2.connect(**db_config)

cursor = conn.cursor()
n = 1

# Itere sobre as linhas do DataFrame e insira os dados na tabela "tiny_products"
for index, row in tiny_estoque_df.iterrows():
    print(f'Loop nº: {n}')
    query = """
    INSERT INTO tiny_stock (tiny_id,nome,sku_tiny,unidade,saldo_reservado,deposito_nome,deposito_desconsiderar,
    deposito_saldo,deposito_empresa)
    VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s);
    """
      
    values = (
        row['id'],
        row['nome'],
        row['codigo'],
        row['unidade'],
        row['saldoReservado'],
        row['deposito.nome'],
        row['deposito.desconsiderar'],
        row['deposito.saldo'],
        row['deposito.empresa'],
    )
    # print(values)
    n += 1
    cursor.execute(query, values)

# Faça o commit para salvar as alterações no banco de dados
conn.commit()

# Feche o cursor e a conexão
cursor.close()
conn.close()
print('Produtos adicionados ao Banco de dados.')


Loop nº: 1
Loop nº: 2
Loop nº: 3
Loop nº: 4
Loop nº: 5
Loop nº: 6
Loop nº: 7
Loop nº: 8
Loop nº: 9
Loop nº: 10
Loop nº: 11
Loop nº: 12
Loop nº: 13
Loop nº: 14
Loop nº: 15
Loop nº: 16
Loop nº: 17
Loop nº: 18
Loop nº: 19
Loop nº: 20
Loop nº: 21
Loop nº: 22
Loop nº: 23
Loop nº: 24
Loop nº: 25
Loop nº: 26
Loop nº: 27
Loop nº: 28
Loop nº: 29
Loop nº: 30
Loop nº: 31
Loop nº: 32
Loop nº: 33
Loop nº: 34
Loop nº: 35
Loop nº: 36
Loop nº: 37
Loop nº: 38
Loop nº: 39
Loop nº: 40
Loop nº: 41
Loop nº: 42
Loop nº: 43
Loop nº: 44
Loop nº: 45
Loop nº: 46
Loop nº: 47
Loop nº: 48
Loop nº: 49
Loop nº: 50
Loop nº: 51
Loop nº: 52
Loop nº: 53
Loop nº: 54
Loop nº: 55
Loop nº: 56
Loop nº: 57
Loop nº: 58
Loop nº: 59
Loop nº: 60
Loop nº: 61
Loop nº: 62
Loop nº: 63
Loop nº: 64
Loop nº: 65
Loop nº: 66
Loop nº: 67
Loop nº: 68
Loop nº: 69
Loop nº: 70
Loop nº: 71
Loop nº: 72
Loop nº: 73
Loop nº: 74
Loop nº: 75
Loop nº: 76
Loop nº: 77
Loop nº: 78
Loop nº: 79
Loop nº: 80
Loop nº: 81
Loop nº: 82
Loop nº: 83
Loop nº: 84
L